In [7]:
import tensorflow as tf

In [8]:
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
from data_utils import get_CIFAR10_data
import resource
import numpy as np
from numpy import float32

# reload(data_utils)
%matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [45]:
def batch_norm(x, n_out, phase_train, scope='bn'):
    """
    Batch normalization on convolutional maps.
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope(scope):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [44]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def conv2d_relu(x, W, b):
    return tf.nn.relu(conv2d(x,W) + b)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def weight_variable(shape, name):    
    initial = tf.truncated_normal(shape, stddev=1.0/math.sqrt(float(1024)), name=name)


def bias_variable(shape, name):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial, name=name)


In [43]:
#todo: calculate size reduced after maxpool

# num_epochs = 200
# num_train = 100
# batch_size = 50
# num_classes = 10

num_epochs = 50
num_train = 5 #49000
batch_size = 5
num_classes = 10

small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}
X = small_data['X_train'].astype(float32)
X = np.reshape(X, (X.shape[0], -1))
y = small_data['y_train']

X_val = small_data['X_val'].astype(float32)
X_val = np.reshape(X_val, (X_val.shape[0], -1))
y_val = small_data['y_val']


W,H = 32,32
C = 3
input_dim = C * W * H
num_conv1_filters = 32
num_conv2_filters = 32


f_size = 3
fc_hidden_dim = 50

"""
Placeholders
"""
x_ = tf.placeholder(tf.float32, shape=[None, input_dim], name="x_placeholder")
x = tf.reshape(x_, [-1,H,W,C]) # double check if this reshape hasn't messed anything up:
y_ = tf.placeholder(tf.int64, shape=(None), name="y_placeholder")
keep_prob = tf.placeholder(tf.float32, name="keep_prob_placeholder")

"""
Conv layer 1
"""
W_conv1_1 = weight_variable((f_size, f_size, C, num_conv1_filters), name="W_conv1_1")
# XXX: left off here, figure out how to best handle auto generation of W.  num_conv1_filters needs to match with next conv3
W_conv1_2 = weight_variable((f_size, f_size, num_conv1_filters, num_conv1_filters), name="W_conv1_2")
b_1_1 = bias_variable([num_conv1_filters], name='b1_1')
b_1_2 = bias_variable([num_conv1_filters], name='b1_2')

print "conv1"
h_conv_relu_1_1 = conv2d_relu(x, W_conv1_1, b_1_1)
h_conv_relu_1_2 = conv2d_relu(h_conv_relu_1_1, W_conv1_2, b_1_2)
print h_conv_relu_1_2
h_pool = max_pool_2x2(h_conv_relu_1_2)
print "--"
# print h_pool

"""
Conv layer 2
"""
W_conv2_1 = weight_variable((f_size, f_size, num_conv1_filters, num_conv2_filters), name="W_conv2_1")
W_conv2_2 = weight_variable((f_size, f_size, num_conv2_filters, num_conv2_filters), name="W_conv2_2")
b_2_1 = bias_variable([num_conv2_filters], name='b2_1')
b_2_2 = bias_variable([num_conv2_filters], name='b2_2')

h_conv_relu_2_1 = conv2d_relu(h_pool, W_conv2_1, b_2_1)
h_conv_relu_2_2 = conv2d_relu(h_conv_relu_2_1, W_conv2_2, b_2_2)
h_pool_2 = max_pool_2x2(h_conv_relu_2_2)
print "conv2"
print h_conv_relu_2_1
print h_conv_relu_2_2
print h_pool_2
print "--"

"""
FC Layer
"""
# double check the 16 number
# W_fc1 = tf.Variable(1e-3*tf.truncated_normal((16*16*num_conv1_filters, fc_hidden_dim), stddev=0.1), name="W_fc1")
W_fc1 = weight_variable((8*8*num_conv2_filters, fc_hidden_dim), name="W_fc1")
b_fc1 = bias_variable([fc_hidden_dim], name="b_fc1")
# h_pool_flat = tf.reshape(h_pool, [-1, 16*16*32])
h_pool_flat = tf.reshape(h_pool_2, [-1, 8*8*num_conv2_filters])
print h_pool_flat
print W_fc1
# (10,8192) * (8192,500) = (10,500)
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
print h_fc1

"""
Dropout
"""
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

"""
Output Layer
"""
W_fc2 = weight_variable((fc_hidden_dim,num_classes), name="W_fc2")
b_fc2 = tf.Variable(tf.zeros([num_classes]), name="b_fc2")
scores_fc2 = (tf.matmul(h_fc1, W_fc2) + b_fc2)
print scores_fc2
# y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)
# print y_conv
# print tf.log(y_conv)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))


cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(scores_fc2, y_, name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
print cross_entropy
print loss

# optimizer = tf.train.GradientDescentOptimizer(1e-2)
# # Create a variable to track the global step.
# global_step = tf.Variable(0, name='global_step', trainable=False)
# train_op = optimizer.minimize(loss, global_step=global_step)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

with tf.Session() as session:
    session.run(tf.initialize_all_variables())

    for epoch in xrange(num_epochs):
        for i in xrange(num_train/batch_size):
            batch_xs = X[i*batch_size:i*batch_size+batch_size] # (batchsize, image_dim)
            batch_ys = y[i*batch_size:i*batch_size+batch_size] # (batchsize,)
#             print i*batch_size, i*batch_size+batch_size

            feed_train = {x_: batch_xs, y_: batch_ys, keep_prob:0.5}
#             print session.run(loss, feed_dict=feed_train)
            session.run(train_op, feed_dict=feed_train)
            
        print session.run(loss, feed_dict=feed_train)
        
        feed_train = {x_: batch_xs, y_: batch_ys, keep_prob:1}
        feed_val = {x_: X_val, y_: y_val, keep_prob:1}
        
        correct_prediction = tf.equal(tf.argmax(scores_fc2, 1), y_)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_acc_result = accuracy.eval(feed_train)
        vac_acc_results = accuracy.eval(feed_val)
        print("(Epoch: {e}/{et}) train acc: {tacc}; val_acc: {vacc}".format(
                e=epoch+1,et=num_epochs, tacc=train_acc_result, vacc=vac_acc_results))

    

conv1
Tensor("Relu_156:0", shape=(?, 32, 32, 32), dtype=float32)
--
conv2
Tensor("Relu_157:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("Relu_158:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("MaxPool_65:0", shape=(?, 8, 8, 32), dtype=float32)
--
Tensor("Reshape_66:0", shape=(?, 2048), dtype=float32)
Tensor("Relu_159:0", shape=(?, 500), dtype=float32)
Tensor("add_192:0", shape=(?, 10), dtype=float32)
Tensor("xentropy_32:0", shape=(?,), dtype=float32)
Tensor("xentropy_mean_32:0", shape=(), dtype=float32)
2.30134
(Epoch: 1/50) train acc: 0.40000000596; val_acc: 0.104999996722
2.29967
(Epoch: 2/50) train acc: 0.40000000596; val_acc: 0.104999996722
2.29698
(Epoch: 3/50) train acc: 0.40000000596; val_acc: 0.104999996722
2.29064
(Epoch: 4/50) train acc: 0.40000000596; val_acc: 0.104999996722
2.25543
(Epoch: 5/50) train acc: 0.40000000596; val_acc: 0.104999996722
2.08824
(Epoch: 6/50) train acc: 0.40000000596; val_acc: 0.104999996722
1.64547
(Epoch: 7/50) train acc: 0.40000000596; val_